In [15]:
'Notebook um sich die Daten genauer anzuschauen'
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from wettbewerb import load_references, get_3montages
import mne
from scipy import signal as sig
import preprocess
import importlib
import features

In [12]:
train_folder = "../shared_data/training_mini"

In [13]:
ids, channels, data, sampling_frequencies, reference_systems, eeg_labels = load_references(train_folder)

100	 Dateien wurden geladen.


In [14]:
reference_channels = channels[0]  # Use the first sample as reference
all_same = True  # Flag to track if all are the same

for i, ch in enumerate(channels):
    if ch != reference_channels:
        all_same = False
        print(f"Sample ID {ids[i]} has different channels.")
        extra = set(ch) - set(reference_channels)
        missing = set(reference_channels) - set(ch)
        if extra:
            print(f"  Extra channels: {extra}")
        if missing:
            print(f"  Missing channels: {missing}")

if all_same:
    print("Every sample has the same channels.")
    print(reference_channels)
    print(len(reference_channels))

Every sample has the same channels.
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
19
